In [ ]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import matplotlib.pyplot as plt
import os, sys, subprocess
%load_ext watermark

In [ ]:
%watermark

In [ ]:
#parameters
workingdir=r"/home/fvalle/phd/master_thesis/docker"
execdir = r"/home/fvalle/phd/master_thesis/docker"
#dockerurl = "cloudadm@193.144.35.220:/home/cloudadm/drive"
#dockerurl = "filippo.valle@c3s.unito.it:"

In [ ]:
os.chdir(workingdir)

# Get the data
[Table_creation](Table_creation.ipynb) does this for you

# Get annotations

In [ ]:
df_file=pd.read_csv("%s/files.dat"%workingdir, index_col=[0])
df_file.head()

In [ ]:
df_file.groupby(by='primary_site').count()

In [ ]:
pc=pd.read_csv("%s/genes.txt"%execdir, index_col=[0])
pc=pc[pc['type_of_gene']=='protein-coding']
pc.head()

# Run statistics
```python
import tacos
tacos.statistics()
tacos.shuffleLabels()
```

CHECK **BioParameters.h** before compiling

In [ ]:
df=pd.read_csv("mainTable.csv", index_col=0,header=0)

In [ ]:
os.system("cp %s/data_mining %s/"%(execdir,workingdir))
os.system("cp %s/addo.py %s/"%(execdir,workingdir))
os.system("./data_mining 0")
os.system("./data_mining 2")

In [ ]:
os.chdir(workingdir)
mv = pd.read_csv("meanVariances.csv", index_col=0)
o=pd.read_csv("O.dat", header=None)
mv.insert(3,'occurrence',o.values)
mv.to_csv("meanVariances.csv")

In [ ]:
fig=plt.figure()
plt.plot(np.sort(np.genfromtxt("A.dat"))[::-1])
plt.xscale('log')
plt.yscale('log')
plt.xlabel("rank, i", fontsize=20)
plt.ylabel("$f_i$", fontsize=20)
plt.tick_params(labelsize=18)
fig.savefig("Zipf.pdf")

In [ ]:
df_heaps=pd.read_csv("heaps.dat", index_col=None, header=None)
df_heaps.columns=['size','diffwords']

In [ ]:
fig=plt.figure()
plt.scatter(df_heaps['size'], df_heaps['diffwords'])
plt.xlim(df_heaps['size'].min()*0.9, df_heaps['size'].max()*1.1)
plt.ylim(df_heaps['diffwords'].min()*0.99, df_heaps['diffwords'].max()*1.01)
plt.xlabel("vocabulary size", fontsize=20)
plt.ylabel("# of different words", fontsize=20)
plt.tick_params(labelsize=18)
fig.savefig("heaps.pdf")

# Analyse table and select genes

[Table_Analyser.ipynb](Table_Analyzer.ipynb)

In [ ]:
os.chdir(execdir)
from tableanalyser import scalinglawsandoverexpressed

In [ ]:
scalinglawsandoverexpressed(workingdir, distance=10, how_many_sigmas=1)

In [ ]:
os.chdir(workingdir)
os.system("mv mainTable.csv mainTable_all.csv")
os.system("mv mainTable_over.csv mainTable.csv")

# Make bipartite network
```bash
python addo.py
```
```python
import tacos
tacos.makegraph()
```

In [ ]:
os.chdir(workingdir)
os.system("./data_mining 0")
os.system("./data_mining 2")
os.system("python addo.py")
os.system("./data_mining 7")

In [ ]:
#os.system("scp graph.xml.gz %s/."%dockerurl)

[hSBM.ipynb]()

# Run stochastic block model
```
docker run -it -p8888:8888 -u user -v $(PWD)/drive/:/home/user/ fvalle01/hsbm bash
jupyter notebook --ip0.0.0.0 --allow-root
```

download the data

In [ ]:
os.chdir(workingdir)
os.system("mkdir -p topsbm")
#os.system("scp %s/* topsbm/."%dockerurl)

# Run cluster analysis

In [ ]:
os.chdir(execdir)
from hsbmpy import clusteranalysis, get_scores, add_score_lines, getclustersizesarray
from geneontology import topicanalysis

In [ ]:
os.chdir(execdir)
import hsbmpy, importlib
importlib.reload(hsbmpy)
from hsbmpy import clusteranalysis, get_scores, add_score_lines, getclustersizesarray
os.chdir(workingdir)

In [ ]:
L_max=np.array([el.split("_")[2] for el in os.listdir("%s/topsbm"%workingdir) if "level_" in el], dtype=int).max()

In [ ]:
labels = df_file.columns
clusteranalysis(workingdir, labels, l=L_max)

In [ ]:
scores = get_scores(workingdir,labels, l=L_max, verbose=False)
fig=plt.figure(figsize=(12,8))
ax = fig.subplots(1)
add_score_lines(ax,scores,labels=[k for k in scores.keys()],xl=getclustersizesarray(workingdir,l=L_max))
plt.show()
fig.savefig("%s/metric.pdf"%(workingdir))

In [ ]:
geneontology.topicanalysis()

# Clean

In [ ]:
os.chdir(workingdir)
os.system("rm -rf data_mining addo.py")